In [1]:
from reader import read, read_url
from grouper import RegionProfile
from color_maker import ColorMaker
from ipywidgets import widgets

rec_url = "https://portal.bluebrain.epfl.ch/wp-content/uploads/2019/02/white_matter_FULL_RECIPE_v1p15.yaml_-3.zip"
rec_fn = "white_matter_FULL_RECIPE_v1p15.yaml"

data, layer_labels = read_url(rec_url, rec_fn)
default_target_region = "SSp-ll"
default_grouping = [("Without layers", "By hemisphere"),
                    ("With layers", "All together"),
                    ("Without layers", "By projection class"),
                    ("Without layers", "By region")]
cols = ColorMaker.factory()

r = [RegionProfile(data[default_target_region], layer_labels, colors=cols)]
valid_grouping = r[0]._types_of_groups + ["None"]
layer_handling = ["Without layers", "With layers"]

In [2]:
region_selector = widgets.Dropdown(description='Target Region', value=default_target_region, options=sorted(data.keys()))
grp_selectors = [widgets.Dropdown(description='{0} grouping'.format(descr),
                                  value=_grouping[1],
                                  options=valid_grouping)
                for descr, _grouping in zip(['First', 'Second', 'Third', 'Fourth'], default_grouping)]
l_selectors = [widgets.Checkbox(description="By target layer",
                                value=_grouping[0] == layer_handling[1])
               for _grouping in default_grouping]
thresh_selector = widgets.FloatSlider(description="Threshold (*1E-3/um^3)", min=0.0, max=100, value=0.0, step=1.0)

r[0].set_up_groups([_x for _x in default_grouping])
fig = r[0].plot()

def full_update():
    new_labels, offset, lbl_colors = r[0].get_plotly_labels()
    link_dict = r[0].get_plotly_links(offset, lbl_colors, threshold=thresh_selector.value/1000)
    with fig.batch_update():
        fig.data[0].node.label = new_labels
        fig.data[0].node.color = lbl_colors
        fig.data[0].node.update()
        fig.data[0].link.update(link_dict)
    fig.update_layout()

def update_links(change):
    _, offset, lbl_colors = r[0].get_plotly_labels()
    link_dict = r[0].get_plotly_links(offset, lbl_colors, threshold=thresh_selector.value/1000)
    with fig.batch_update():
        fig.data[0].link.update(link_dict)

def update_region(change):
    r.pop()
    r.append(RegionProfile(data[region_selector.value], layer_labels, colors=cols))
    update_grouping(change)

def update_grouping(change):
    new_grouping = [(layer_handling[c.value], g.value)
                    for g, c in zip(grp_selectors, l_selectors)
                   if g.value != "None"]
    r[0].set_up_groups(new_grouping)
    full_update()
    

thresh_selector.observe(update_links, names="value")
region_selector.observe(update_region, names="value")
[_selector.observe(update_grouping, names="value") for _selector in grp_selectors]
[_selector.observe(update_grouping, names="value") for _selector in l_selectors]
grp_box = widgets.VBox(list(map(widgets.HBox, zip(grp_selectors, l_selectors))))
widgets.VBox([
    widgets.HBox([
        widgets.VBox([region_selector, thresh_selector]),
        grp_box
    ]),
    fig
])

                              